## Reacciones

Aquí se encuentra la función que calcula la contribución de las reacciones al vector dndt (variación del nro de partículas en función del tiempo) y todas las funciones necesarias para hacer la cuenta. Por lo pronto sólo se consideran las reacciones químicas en las que estén involucradas O y H. El orden considerado en dndt es
n[n_species] =      {H2,       H,      O,      O2,     OH,     H2O,    H2O2,   HO2}

### Importo dependencias

In [1]:
#Importo librerías
import numpy as np

In [2]:
#Código necesario para importar funciones de otros notebooks .ipynb

import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod


class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

#Ejemplo
# from derivada_reacciones import *

In [3]:
#Importo notebooks
from parametros import n_species, n_reacc, Na, Vol, var_termo

importing Jupyter notebook from parametros.ipynb


### Lista de reacciones

In [4]:
def lista_de_reacciones(y, n, r):
    #Para dado t y n, calcula los términos de dndt debido a las reacciones qcas y los carga en r.
    #t: tiempo
    #y: vector con el valor de las variables termodinámicas
    #n: nro de partículas de cada especie
    #r: vector que contiene la contribución de las reacciones químicas. Cada componente corresponde a una reacción
    #Por lo pronto sólo se consideran las reacciones químicas en las que estén involucradas O y H.

    #Asigno los valores de las constantes termodinámicas
    R = y[var_termo["R"]] #Radio de la burbuja
    V = Vol(R) #Volumen de la burbuja
    T = y[var_termo["T"]] #Temperatura dentro de la burbuja

    ntot = 0.0
    for i in range(n_species):
        ntot = ntot + n[i]

    MolCov=5.1e-29; #molecular covolumen [m^3]
    VolExcl=np.exp(ntot/V*MolCov/(1.0-ntot/V*MolCov))/(1.0-ntot/V*MolCov)

    #Parametros de disociacion.
    #Son empleados para calcular los cambios debido a las reacciones. Creo que los que no están presentados son todos 0.
    t1=1.0
    t2=1.0
    t5=1.0
    t6=1.0
    t15=-1.0
    t16=1.0
    t33=-1.0
    t34=-1.0
    t39=1.0

    #Cargo las reacciones: par es forward, impar es backwards.

    #Reacción 1: O + O + M -> O2 + M
    r[0]=pow(VolExcl,t1)*1.2e17*(1.0e-12)*(ntot/V)*(n[2]/V)*(n[2]/V)*pow(T,-1.0)*np.exp(-0.0/T)/Na/Na
    r[1]=3.16e19*(1.0e-6)*(ntot/V)*(n[3]/V)*pow(T,-1.3)*np.exp(-59893.0/T)/Na

    #Reacción 2: O + H + M -> OH + M
    r[2]=pow(VolExcl,t2)*5.0e17*(1.0e-12)*(ntot/V)*(n[2]/V)*(n[1]/V)*pow(T,-1.0)*np.exp(-0.0/T)/Na/Na
    r[3]=3.54e17*(1.0e-6)*(ntot/V)*(n[4]/V)*pow(T,-0.9)*np.exp(-51217.0/T)/Na

    #Reacción 3: O + H2 -> H + OH
    r[4]=3.87e4*(1.0e-6)*(n[2]/V)*(n[0]/V)*pow(T,2.7)*np.exp(-3150.0/T)/Na
    r[5]=1.79e4*(1.0e-6)*(n[1]/V)*(n[4]/V)*pow(T,2.7)*np.exp(-2200.0/T)/Na

    #Reacción 4: H + O2 -> O +OH
    r[6]=2.65e16*(1.0e-6)*(n[1]/V)*(n[3]/V)*pow(T,-0.7)*np.exp(-8576.0/T)/Na
    r[7]=9.0e13*(1.0e-6)*(n[2]/V)*(n[4]/V)*pow(T,-0.3)*np.exp(83.0/T)/Na

    #Reacción 5: H + H + M -> H2 + M
    r[8]=pow(VolExcl,t5)*1.0e18*(1.0e-12)*(ntot/V)*(n[1]/V)*(n[1]/V)*pow(T,-1.0)*np.exp(-0.0/T)/Na/Na
    r[9]=7.46e17*(1.0e-6)*(ntot/V)*(n[0]/V)*pow(T,-0.8)*np.exp(-52177.0/T)/Na

    #Reacción 6: H + OH + M -> H2O + M
    r[10]=pow(VolExcl,t6)*2.2e22*(1.0e-12)*(ntot/V)*(n[1]/V)*(n[4]/V)*pow(T,-2.0)*np.exp(-0.0/T)/Na/Na
    r[11]=3.67e23*(1.0e-6)*(ntot/V)*(n[5]/V)*pow(T,-2.0)*np.exp(-59980.0/T)/Na

    #Reacción 7: OH + H2 -> H + H2O
    r[12]=2.16e8*(1.0e-6)*(n[4]/V)*(n[0]/V)*pow(T,1.5)*np.exp(-1726.0/T)/Na
    r[13]=5.2e9*(1.0e-6)*(n[1]/V)*(n[5]/V)*pow(T,1.3)*np.exp(-9529.0/T)/Na

    #Reacción 8: OH + OH -> O + H2O
    r[14]=3.57e4*(1.0e-6)*(n[4]/V)*(n[4]/V)*pow(T,2.4)*np.exp(-1062.0/T)/Na
    r[15]=1.74e6*(1.0e-6)*(n[2]/V)*(n[5]/V)*pow(T,2.2)*np.exp(-7693.0/T)/Na

    #Reacción 46: antes numeradas 90 y 91: HO2 + HO2 -> H2O2 + O2
    r[16]=3.0e12*(1.0e-6)*(n[7]/V)*(n[7]/V)*pow(T,0.0)*np.exp(-700.0/T)/Na
    r[17]=4.53e14*(1.0e-6)*(n[6]/V)*(n[3]/V)*pow(T,-0.39)*np.exp(-19700.0/T)/Na

    #Reacción 47: antes numeradas 92 y 93: H2O2 -> OH + OH
    r[18]=1.2e17*(1.0e-6)*(n[6]/V)*(ntot/V)*pow(T,0.0)*np.exp(-22900.0/T)/Na
    r[19]=9.0e5*(1.0e-12)*(ntot/V)*(n[4]/V)*(n[4]/V)*pow(T,0.9)*np.exp(3050.0/T)/Na/Na

    #Reacción 48: antes numeradas 94 y 95: H2O2 + H -> H2O + OH
    r[20]=3.2e14*(1.0e-6)*(n[6]/V)*(n[1]/V)*pow(T,0.0)*np.exp(-4510.0/T)/Na
    r[21]=1.14e9*(1.0e-6)*(n[5]/V)*(n[4]/V)*pow(T,1.36)*np.exp(-38180.0/T)/Na

    #Reacción 49: antes numeradas 96 y 97: H2O2 + H -> H2 + HO2
    r[22]=4.82e13*(1.0e-6)*(n[6]/V)*(n[1]/V)*pow(T,0.0)*np.exp(-4000.0/T)/Na
    r[23]=1.41e11*(1.0e-6)*(n[0]/V)*(n[7]/V)*pow(T,0.66)*np.exp(-12320.0/T)/Na

    #Reacción 50: antes numeradas 98 y 99: H2O2 + O -> OH + HO2
    r[24]=9.55e6*(1.0e-6)*(n[6]/V)*(n[2]/V)*pow(T,2.0)*np.exp(-2000.0/T)/Na
    r[25]=4.62e3*(1.0e-6)*(n[4]/V)*(n[7]/V)*pow(T,2.75)*np.exp(-9277.0/T)/Na

    #Reacción 51: antes numeradas 100 y 101: H2O2 + OH -> H2O + HO2
    r[26]=1.00e13*(1.0e-6)*(n[6]/V)*(n[4]/V)*pow(T,0.0)*np.exp(-900.0/T)/Na
    r[27]=2.8e13*(1.0e-6)*(n[5]/V)*(n[7]/V)*pow(T,0.0)*np.exp(-16500.0/T)/Na


In [5]:

def derivada_reacc_qcas(y, n, dndt):
    #Calcula el efecto de todas las reacciones qcas sobre la derivada dndt


    #Creo el vector que contendrá el efecto de cada reacción
    r = np.zeros(n_reacc)
    lista_de_reacciones(y, n, r)

    #Asigno los valores de las constantes termodinámicas
    R = y[var_termo["R"]] #Radio
    V = Vol(R) #Volumen

    dndt[0] = dndt[0] + V*(-1.0*(r[4]-r[5])+1.0*(r[8]-r[9])-1.0*(r[12]-r[13])+1.0*(r[22]-r[23]))
    dndt[1] = dndt[1] + V*(-1.0*(r[2]-r[3])+1.0*(r[4]-r[5])-1.0*(r[6]-r[7])-2.0*(r[8]-r[9])-1.0*(r[10]-r[11])+1.0*(r[12]-r[13])-1.0*(r[20]-r[21])-1.0*(r[22]-r[23]))
    dndt[2] = dndt[2] + V*(-2.0*(r[0]-r[1])-1.0*(r[2]-r[3])-1.0*(r[4]-r[5])+1.0*(r[6]-r[7])+1.0*(r[14]-r[15])-1.0*(r[24]-r[24]))
    dndt[3] = dndt[3] + V*(1.0*(r[0]-r[1])-1.0*(r[6]-r[7])+1.0*(r[16]-r[17]))
    dndt[4] = dndt[4] + V*(+1.0*(r[2]-r[3])+1.0*(r[4]-r[5])+1.0*(r[6]-r[7])-1.0*(r[10]-r[11])-1.0*(r[12]-r[13])-2.0*(r[14]-r[15])+2.0*(r[18]-r[19])+1.0*(r[20]-r[21])+1.0*(r[24]-r[24]))
    dndt[5] = dndt[5] + V*(+1.0*(r[10]-r[11])+1.0*(r[12]-r[13])+1.0*(r[14]-r[15])+ 1.0*(r[20]-r[21])+1.0*(r[26]-r[27]))
    dndt[6] = dndt[6] + V*(-1.0*(r[18]-r[19])-1.0*(r[20]-r[21])-1.0*(r[22]-r[23])-1.0*(r[24]-r[24])-1.0*(r[26]-r[27])+1.0*(r[16]-r[17]))
    dndt[7] = dndt[7] + V*(-2.0*(r[16]-r[17])+1.0*(r[22]-r[23])+1.0*(r[24]-r[24])+1.0*(r[26]-r[27]))

    return

In [14]:
#Código viejo: antes podía configurar reacciones_qcas para que solo tenga en cuenta algunas reacciones. Usaba dentro de esa función el siguiente código
    # //1 reacción:
    # dndt[0]=0
    # dndt[1]=0
    # dndt[2]=V(t)*(-2.0*(r[0]-r[1]))
    # dndt[3]=V(t)*(1.0*(r[0]-r[1]))
    # dndt[4]=0

    # //3 reacciones:
    # dndt[0]=V(t)*(-1.0*(r[4]-r[5]))
    # dndt[1]=V(t)*(-1.0*(r[2]-r[3])+1.0*(r[4]-r[5]))
    # dndt[2]=V(t)*(-2.0*(r[0]-r[1])-1.0*(r[2]-r[3])-1.0*(r[4]-r[5]))
    # dndt[3]=V(t)*(1.0*(r[0]-r[1]))
    # dndt[4]=V(t)*(+1.0*(r[2]-r[3])+1.0*(r[4]-r[5]))

    # //Todas las reacciones que no involucran N (Nitrógeno):
    # //n[0] = y[Nvar2+1] =H2.
    # //n[1] = y[Nvar2+2] =H
    # //n[2] = y[Nvar2+3] =O
    # //n[3] = y[Nvar2+4] =O2
    # //n[4] = y[Nvar2+5] =OH
    # //n[5] = y[Nvar2+6] =H2O = y[4] = vapor
    # //n[6] = y[Nvar2+18] =H2O2. Este es mi n[6]
    # //n[7] = y[Nvar2+19] =HO2